In [ ]:
from __future__ import division
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16.0, 16.0)
import cv2
import time

from mpl_toolkits.mplot3d import Axes3D

In [ ]:
from CameraNetwork import Gimbal
p = Gimbal()

In [ ]:
import itertools
for x, y in itertools.product(np.linspace(0, 180, 6), np.linspace(0, 180, 6)):
    p.move(int(x), int(y))
    time.sleep(2)

In [ ]:
p.move(170, 90)

In [ ]:
import ids

In [ ]:
class camera(object):
    def __init__(self):
        self._cam = ids.Camera(nummem=1)

        self._cam.auto_white_balance = False
        self._cam.continuous_capture = False
        self._cam.auto_exposure = False
        self._cam.exposure = 100    
        self._cam.color_mode = ids.ids_core.COLOR_BAYER_8
        self._cam.gain_boost = False
        self._cam.gain_db = 0
        
    def measure(self, exposure=40, gain_db=0, average=10):
        self._cam.exposure = exposure
        self._cam.gain_db = gain_db
        
        self._cam.continuous_capture = True
        _, _ = self._cam.next()
        
        imgs = []
        for i in range(average):
            img_array, meta_data = self._cam.next()
            imgs.append(img_array)

        self._cam.continuous_capture = False
        
        return np.round(np.mean(imgs, axis=0), 0).astype(np.int32)

def RGBsplit(img):
    R = np.ascontiguousarray(img[::2, ::2])
    B = np.ascontiguousarray(img[1::2, 1::2])
    G1 = np.ascontiguousarray(img[1::2, 0::2])
    G2 = np.ascontiguousarray(img[0::2, 1::2])
    
    return R, ((G1+G2)/2).astype(R.dtype), B


def findSpot(img, threshold=10):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution by calculating its
    moments"""
    
    img_filt = img.copy().astype(np.float)
    img_filt[img<threshold] = 0
    
    total = img_filt.sum()
    Y, X = np.indices(img.shape)
    
    x = (X*img_filt).sum()/total
    y = (Y*img_filt).sum()/total
    
    #col = data[:, int(y)]
    #width_x = np.sqrt(np.abs((np.arange(col.size)-y)**2*col).sum()/col.sum())
    #row = data[int(x), :]
    #width_y = np.sqrt(np.abs((np.arange(row.size)-x)**2*row).sum()/row.sum())
    
    R, G, B = RGBsplit(img_filt)
    
    return int(x), int(y), R.sum()/(R>0).sum(), G.sum()/(G>0).sum(), B.sum()/(B>0).sum()

In [ ]:
try:
    del cam
    time.sleep(5)
except:
    pass
cam = camera()

In [ ]:
img = cam.measure()
#findSpot(img)

In [ ]:
i = img[1:100:2, 751:850:2]

In [ ]:
def RGBsplit(img):
    R = np.ascontiguousarray(img[::2, ::2])
    B = np.ascontiguousarray(img[1::2, 1::2])
    G1 = np.ascontiguousarray(img[1::2, 0::2])
    G2 = np.ascontiguousarray(img[0::2, 1::2])
    
    return R, ((G1+G2)/2).astype(R.dtype), B


def meanColor(c):
    
    nnz_total = (c>0).sum()
    if nnz_total == 0:
        return 0
    
    return c.sum()/nnz_total

In [ ]:
p.move(170, 90)
time.sleep(1)
img = cam.measure()

In [ ]:
threshold = img.max()/2
kernel = np.ones((3, 3),np.uint8)
mask = (img>threshold)
mask = cv2.dilate(mask.astype(np.uint8), kernel)    
#mask = cv2.erode(mask.astype(np.uint8), kernel)    
img_filt = img.astype(np.float) * mask

print [meanColor(c) for c in RGBsplit(img_filt)]
plt.imshow(img_filt[61:100:2, 791:850:2], interpolation='nearest')

In [ ]:
mask = (img>threshold)
mask = cv2.dilate(mask.astype(np.uint8), kernel)    
mask = cv2.erode(mask.astype(np.uint8), kernel, iterations=2)    
img_filt = img.astype(np.float) * mask
plt.imshow(img_filt[61:100:2, 791:850:2], interpolation='nearest')
print [meanColor(c) for c in RGBsplit(img_filt)]

In [ ]:
p.move(90, 90)
time.sleep(1)
img = cam.measure()

In [ ]:
threshold = img.max()/2
kernel = np.ones((3, 3),np.uint8)
mask = (img>threshold)
mask = cv2.dilate(mask.astype(np.uint8), kernel)    
#mask = cv2.erode(mask.astype(np.uint8), kernel)    
img_filt = img.astype(np.float) * mask

print [meanColor(c) for c in RGBsplit(img_filt)]
plt.imshow(img_filt[601:700:2,801:900:2], interpolation='nearest')
#plt.imshow(img_filt[61:100:2, 791:850:2], interpolation='nearest')

In [ ]:
mask = (img>threshold)
mask = cv2.dilate(mask.astype(np.uint8), kernel)    
mask = cv2.erode(mask.astype(np.uint8), kernel, iterations=2)    
img_filt = img.astype(np.float) * mask
plt.imshow(img_filt[601:700:2,801:900:2], interpolation='nearest')
print [meanColor(c) for c in RGBsplit(img_filt)]

In [ ]:
from scipy import interpolate

In [ ]:
img_rgb = np.load(r'C:\Users\amitibo\Documents\code\cameranetwork\scripts_calibrate\test.npy')
#img = img[1::2, ::2]

In [ ]:
img = img_rgb[..., 2]
y, x = np.nonzero(img)
z = img[np.nonzero(img)]

In [ ]:
plt.scatter(x, y, s=z)

In [ ]:
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x_, y_, z_)

In [ ]:
%matplotlib inline

In [ ]:
thresh = 150
x_ = x[z>thresh]
y_ = y[z>thresh]
z_ = z[z>thresh]

In [ ]:
from numpy.polynomial import polynomial
def polyfit2d(x, y, f, deg):
    x = np.asarray(x)
    y = np.asarray(y)
    f = np.asarray(f)
    deg = np.asarray(deg)
    vander = polynomial.polyvander2d(x, y, deg)
    vander = vander.reshape((-1,vander.shape[-1]))
    f = f.reshape((vander.shape[0],))
    c = np.linalg.lstsq(vander, f)[0]
    return c.reshape(deg+1)

In [ ]:
c = polyfit2d(x_, y_, z_, [2, 2])

In [ ]:
Y, X = np.mgrid[0:1200, 0:1600]
f = polynomial.polyval2d(X, Y, c)

In [ ]:
znew = np.clip(f, 0, 10000)
plt.imshow(znew, cmap='gray')

In [ ]:
Y, X = np.mgrid[0:1200:10, 0:1600:10]
f = polynomial.polyval2d(X, Y, c)
znew = np.clip(f, 9, 1000)
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, znew)
ax.scatter(x, y, z)

In [ ]:
p.move(50, 90)

In [ ]:
%matplotlib inline

In [ ]:
z_estim = polynomial.polyval2d(x, y, c)
znew = np.clip(f, 9, 1000)
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z-z_estim)

In [ ]:
z.shape

In [ ]:
znew.shape

In [ ]:
x.shape